## Data Inspection

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Import dataset
df = pd.read_csv("../raw_data.csv")

In [ ]:
# Inspect dataset
df.shape

In [ ]:
df.info()

In [ ]:
df.value_counts("zip_code")

In [ ]:
df.value_counts("commune")

In [ ]:
df.value_counts("province")

In [ ]:
df.value_counts("garden")

In [ ]:
df.value_counts("bedroom_nr")

In [ ]:
df.value_counts("building_condition")

In [ ]:
df.value_counts("plot_surface")

In [ ]:
df.describe()

## Data Cleaning

### Notes on missing values and zero values:

**Missing Values:**

* "price" column: 36 entries missing                **-> Drop (Alek) DONE**
* "building_condition": 6690 entries missing        **-> Encode as "Unknown" -> Alek -> DONE**
* "facade_number": 9362 entries missing             **-> Use median based on subtype of property! -> Me -> DONE**

**Zero Values:**

* "living_area": 5 entries with 0                   **-> Drop -> Me -> DONE**
* "equipped_kitchen":                               **-> 0 means no equipped kitchen   -> OK**
* "bedroom_nr": 890 entries with 0                  **-> Either studio apartments or mixed use buildings etc.  --> OK**
* "garden": 20949 entries with 0                    **-> apr. 20% of df have a garden -> matches filter result in immoweb** 
                                                    **-> in our dataset apr. 50%, but we had 60% houses, here 39% houses**
* "plot_surface": 16290 entries with 0              **-> Definition: based on entry in immoweb -> OK** 


**Other:**
* "terrace":                               **>0 means no terrace, and number = surface?  -> OK**
* "garden":                                **>same as terrace? -> OK**
* "subtype_of_property"                    **> remove 'unit' -> Celina -> DONE**
* "building_condition"                     **> Reduce number of categories -> Alek -> DONE**
* "equipped_kitchen"                       **> reduce number of categories -> Alek -> DONE**

In [ ]:
# See summary of missing values
df.isna().sum()

### Calculate missing value threshold and drop entries below or equal to

In [ ]:
# Calculate missing value threshold
threshold = len(df) * 0.05
print(len(df))
print(int(threshold))

# Use Boolean indexing to filter for columns with missing values <= threshold and > 0
cols_to_drop = df.columns[(df.isna().sum() <= threshold) & (df.isna().sum() > 0)]

print(cols_to_drop)

# To drop missing values
df.dropna(subset=cols_to_drop, inplace=True)

### Drop zero values in "living_area"


In [ ]:
# Drop rows where 'living_area' is 0
df = df[df["living_area"] != 0]

### Handling missing values in "building_condition"

Options:

* Fill with most frequent category (mode) --> No

* Fill with a placeholder (e.g. "Unknown") --> OK

#### Fill zero values in "building_condition" with "Unknown"

### Replace missing values in "facade_number" with median based on subtype of property

In [ ]:
# Compute median facade number by subtype
facade_dict = df.groupby("subtype_of_property")["facade_number"].median().to_dict()

# Impute values
df["facade_number"] = df["facade_number"].fillna(df["subtype_of_property"].map(facade_dict))

In [ ]:
df.isna().sum()

## Check for duplicates

In [ ]:
# Check for duplicates and count them
num_duplicates = df.duplicated().sum()
num_duplicates

In [ ]:
# Remove duplicate rows and create cleaned df
#df_unique = df.drop_duplicates()

### Notes on duplicates:

* 1284 duplicates found --> while scraping duplicates were eliminated, so the 1284 entries are in fact no duplicates

## Check for blank spaces

In [ ]:
# Check for leading/trailing spaces in string columns

columns_with_spaces = [col for col in df.columns if df[col].dtype == 'object' and df[col].str.contains(r'^\s+|\s+$').any()]
columns_with_spaces

In [ ]:
# Check for multiple spaces within a string
columns_with_spaces = [col for col in df.columns if df[col].dtype == 'object' and df[col].str.contains(r'  ').any()]
columns_with_spaces

#### Result: no leading or trailing whitespace and no double whitespace

## Data Cleaning: Must-have features

* No duplicates            -> OK
* No blank spaces          -> OK
* No empty values          -> OK
* No errors                -> I guess

Notes:
* Duplicates: In the scraping code the listing id was checked to avoid duplicates; the entries in the df that appear as duplicates are new buildings that appear identical, but are in fact distinct listings

In [33]:
# Import cleaned dataset
df_cleaned = pd.read_csv("../cleaned-data.csv")